In [1]:
import geopandas
import folium
from folium.plugins import MarkerCluster
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
# Check if running in Google Colab
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    # Define the base path for your data directory for Colab
    basePath = '/content/drive/MyDrive'

else:
    # Running locally or in a different environment
    basePath = '..'

In [3]:
df = pd.read_csv(f'{basePath}/data/processed/unified_countries.csv')

In [4]:
try:
    # Using 'STATION' as the unique identifier for stations
    df_map = df[['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'Source_Country_Name']].dropna(subset=['LATITUDE', 'LONGITUDE']).drop_duplicates(subset=['STATION']).copy()

    # Ensure LATITUDE and LONGITUDE are numeric and handle potential errors
    df_map['LATITUDE'] = pd.to_numeric(df_map['LATITUDE'], errors='coerce')
    df_map['LONGITUDE'] = pd.to_numeric(df_map['LONGITUDE'], errors='coerce')
    df_map.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)

    if not df_map.empty:

        gdf = geopandas.GeoDataFrame(
            df_map,
            geometry=geopandas.points_from_xy(df_map.LONGITUDE, df_map.LATITUDE)
        )

        # Create a base map centered roughly on the world for a global view
        m = folium.Map(location=[0, 0], zoom_start=1.5)

        # Add a marker cluster to group nearby markers, improving performance for many stations
        marker_cluster = MarkerCluster().add_to(m)

        # Iterate through each station in the GeoDataFrame and add a marker to the cluster
        for idx, row in gdf.iterrows():
            folium.Marker(
                location=[row['LATITUDE'], row['LONGITUDE']], # Marker location
                popup=f"Station: {row['NAME']} ({row['STATION']})<br>Country: {row['Source_Country_Name']}", # Popup on click
                tooltip=f"{row['NAME']} - {row['Source_Country_Name']}" # Tooltip on hover
            ).add_to(marker_cluster)

        # Display the map in the notebook output
        display(m)
        print("Map displayed above showing station locations.")
    else:
        print("No valid station coordinates found for mapping after cleaning.")
except Exception as e:
    print(f"An unexpected error occurred during map visualization: {e}")

Map displayed above showing station locations.
